In [1]:
import os
import json

# path = r"G:\My Drive\my_work\temp\instances_train2017.json\instances_train2017.json"


In [2]:
# # open json file
# with open(path, 'r') as f:
#     data = json.load(f)


In [2]:
train_img_folder = r"E:\HD_VNese_map\dataset\training_data\images\train"
val_img_folder = r"E:\HD_VNese_map\dataset\training_data\images\val"
train_anno_folder = r"E:\HD_VNese_map\dataset\training_data\labels\train"
val_anno_folder = r"E:\HD_VNese_map\dataset\training_data\labels\val"



“id”: A unique identifier for the image.
“width”: The width of the image in pixels.
“height”: The height of the image in pixels.
“file_name”: The file name of the image.
“license”: The license associated with the image.

“id”: A unique identifier for the annotation.
“image_id”: The identifier of the image containing the annotated object.
“category_id”: The identifier of the category (i.e., class) of the annotated object.
“bbox”: A list of four numbers representing the bounding box of the annotated object in the format [x, y, width, height], where (x, y) is the top-left corner of the bounding box.
“area”: The area of the bounding box in square pixels.

In [12]:
import cv2
from tqdm import tqdm
import shutil
import PIL
from PIL import Image, ImageSequence
import numpy as np
def convert_gif_to_jpg(img_path):
    # Open the gif image using PIL
    gif = Image.open(f"{img_path}")
    # write only the first frame of the gif to a new file
    for i, frame in enumerate(ImageSequence.Iterator(gif)):
        # Convert PIL image to OpenCV format
        img = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)
        # Save the image
        cv2.imwrite(f"{img_path.replace('.gif', '.jpg')}", img)
        break


def convert_yolo_to_coco(image_folder, anno_folder, save_path):
    train_img_folder = image_folder
    train_anno_folder = anno_folder

    license = "0"
    count_except = 0
    train_json = {
        "info": {
            "description": "HD Vietnamese Map",
            "url": "",
            "version": "1.0",
            "year": 2024,
        },
        "images": [],
        "annotations": [],
        "categories": [
            {
                "id": 0,
                "name": "china",
                "supercategory": "map"
            },
            {
                "id": 1,
                "name": "vietnam",
                "supercategory": "map"
            },
            {
                "id": 2,
                "name": "sea",
                "supercategory": "map"
            },
            {
                "id": 3,
                "name": "_",
                "supercategory": "_"
            },
            {
                "id": 4,
                "name": "nine_dash_line",
                "supercategory": "map"
            },
            {
                "id": 5,
                "name": "flag",
                "supercategory": "flag"
            }
        ],
    }
    except_img_folder = r"E:\HD_VNese_map\dataset\training_data\images\except"
    except_anno_folder = r"E:\HD_VNese_map\dataset\training_data\labels\except"
    if not os.path.exists(except_img_folder):
        os.makedirs(except_img_folder)
    if not os.path.exists(except_anno_folder):
        os.makedirs(except_anno_folder)
    # rename images
    num_train_img = len(os.listdir(train_img_folder))
    train_img_list = os.listdir(train_img_folder)
    unique_id_for_annotation = 0
    for _id in tqdm(range(num_train_img)):
        img_filename = train_img_list[_id]
        if img_filename.endswith(".gif"):
            convert_gif_to_jpg(os.path.join(train_img_folder, img_filename))
            img_filename = img_filename.replace(".gif", ".jpg")

        label_filename = img_filename.replace(".jpg", ".txt")
        img = cv2.imread(os.path.join(train_img_folder, img_filename))
        try:
            h, w, _ = img.shape
        except:
            print("except", img_filename)
            # remove image
            try:
                shutil.move(os.path.join(train_img_folder, img_filename), os.path.join(except_img_folder, img_filename))
            except:
                print("except when moving:", img_filename)
            continue
        # setup img     info
        img_info = {
            "license": license,
            "file_name": img_filename,
            "height": h,
            "width": w,
            "id": _id
        }
        train_json["images"].append(img_info)
        if os.path.exists(os.path.join(train_anno_folder, label_filename)):
            with open(os.path.join(train_anno_folder, label_filename), 'r') as f:
                anno = f.readlines()
                for line in anno:
                    line = line.strip().split()
                    try:
                        x, y, w, h = float(line[1]), float(line[2]), float(line[3]), float(line[4])
                    except:
                        count_except += 1
                        print("except", line, "in", label_filename)
                        # move img and label to except folder
                        # close both files before moving    
                        f.close()
                        try:
                            shutil.move(os.path.join(train_img_folder, img_filename), os.path.join(except_img_folder, img_filename))
                            shutil.move(os.path.join(train_anno_folder, label_filename), os.path.join(except_anno_folder, label_filename))
                        except:
                            print("except when moving:", img_filename)
                            
                        continue
                    # convert to x_left, y_top, w,h 
                    x = x - w/2
                    y = y - h/2
                    # unnormallize
                    x = x * img_info["width"]
                    y = y * img_info["height"]
                    w = w * img_info["width"]
                    h = h * img_info["height"]
                    # setup annotation
                    unique_id_for_annotation += 1
                    anno_info = {
                        "id": unique_id_for_annotation,
                        "image_id": _id,
                        "category_id": int(float(line[0])),
                        "bbox": [x, y, w, h],
                        "area": w * h,
                    }
                    train_json["annotations"].append(anno_info)
    with open(save_path, 'w') as f:
        json.dump(train_json, f)
    print("Done")

convert_yolo_to_coco(train_img_folder, train_anno_folder, r"E:\HD_VNese_map\dataset\training_data\labels\train.json")
convert_yolo_to_coco(val_img_folder, val_anno_folder, r"E:\HD_VNese_map\dataset\training_data\labels\val.json")
            


  0%|          | 0/46743 [00:00<?, ?it/s]

100%|██████████| 46743/46743 [10:14<00:00, 76.09it/s] 


Done


100%|██████████| 10597/10597 [01:35<00:00, 110.74it/s]


Done


In [9]:
ImageSequence.Iterator

                                              0.0/2.6 MB ? eta -:--:--
                                              0.0/2.6 MB ? eta -:--:--
     -                                        0.1/2.6 MB 2.2 MB/s eta 0:00:02
     ----                                     0.3/2.6 MB 2.9 MB/s eta 0:00:01
     ---------                                0.6/2.6 MB 4.9 MB/s eta 0:00:01
     -------------------                      1.3/2.6 MB 7.4 MB/s eta 0:00:01
     --------------------------------------   2.5/2.6 MB 12.4 MB/s eta 0:00:01
     ---------------------------------------- 2.6/2.6 MB 10.4 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 10.0.0
    Uninstalling Pillow-10.0.0:
      Successfully uninstalled Pillow-10.0.0


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Administrator\\AppData\\Local\\Temp\\3\\pip-uninstall-jabcedga\\_imaging.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [10]:
import PIL
print(PIL.__version__)

9.4.0
